In [1]:
import pickle
import os
import re
from collections import Counter
import math
import matplotlib.pyplot as plt
import random
import json
import datetime
import string
string.punctuation += '➡•’'
import numpy as np
from adjustText import adjust_text
import matplotlib.cm as cm
import matplotlib
import spacy
import pandas as pd

In [2]:
nlp=spacy.load('en_core_web_md', disable=['parser', 'tagger', 'ner'])

In [3]:
# sp_tokens = ['de', 'la', 's', 'm', 'las', 'los', 'p', 've', 'en', 're', 'del', 't', 'que' ]""

In [4]:
dem = pickle.load(open('../../data/all_tweet_texts_dem.p', 'rb'))
rep = pickle.load(open('../../data/all_tweet_texts_rep.p', 'rb'))


In [5]:
data_path = '../../data/'
house_rep_screennames = pickle.load(open(data_path+'house_rep_screennames.p', 'rb'))
house_dem_screennames = pickle.load(open(data_path+'house_dem_screennames.p', 'rb'))
senate_dem_screennames = pickle.load(open(data_path+'senate_dem_screennames.p', 'rb'))
senate_rep_screennames = pickle.load(open(data_path+'senate_rep_screennames.p', 'rb'))
govenor_rep_screennames = pickle.load(open(data_path+'govenor_rep_screennames.p', 'rb'))
govenor_dem_screennames = pickle.load(open(data_path+'govenor_dem_screennames.p', 'rb'))

In [6]:
all_screenname = set(house_rep_screennames + house_dem_screennames + senate_dem_screennames + senate_rep_screennames\
                + govenor_rep_screennames + govenor_dem_screennames + ['realDonaldTrump'])

In [7]:
covid_vocab = pickle.load(open('../../data/all_covid_words.p', 'rb'))

In [8]:
big_regex = re.compile('|'.join(map(re.escape, covid_vocab)))

In [9]:
def get_covid_tweets(df):
    all_covid_tweets = []
    all_bg_tweets = []
    for idx, row in df.iterrows():
        try:
            line = row['Text']
            line = line.lower()
            if 'covid' in line or 'coronavirus' in line:
                processed_line = big_regex.sub('covid', line)
                all_covid_tweets.append(processed_line)
            else:
                all_bg_tweets.append(line)
        except Exception as ex:
            continue
        
    return all_covid_tweets, all_bg_tweets

In [10]:
dem_covid, dem_bg = get_covid_tweets(dem)
rep_covid, rep_bg = get_covid_tweets(rep)
bg = dem_bg + rep_bg

/var/folders/9t/lplpnk8j495cb3drsnvfhsfw0000gn/T/ipykernel_18283/2231793725.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  for idx, row in df.iterrows():


In [11]:
dem_covid[0]

'the health of american people &amp; u.s. economy must come first. instead of forcing #unemployment aid to lapse while republicans fight among themselves, senate gop should join dems in extending #ui to save people &amp; businesses as covid cases are surging. \n'

In [12]:
len(dem_covid), len(rep_covid), len(bg)

(61944, 37879, 340416)

In [13]:
exclude_words = set(['Ricketts', 'RepPressley', 'Beshear', 'GregHilburn1', 'HouseDemocrats', 'RepBera', 'OversightDems', 
 'WaysandMeansGOP', 'dougducey', 'SteveScalise', 'HouseGOP', 'SecretarySonny', 'Hutchinson', 'GOPoversight',
 'AsaHutchinson','GOPLeader', 'GovRicketts','WhiteHouse','m', 'ChadSabadie', 'HouseCommerce','✓'])

In [14]:
punc = string.punctuation.replace("#", '')

In [15]:
punc

'!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~➡•’'

In [16]:
def preprocess_text(dem, rep, bg):
    print(dem[0])
    dem = [t.replace('\n', '').replace('&amp', 'and').translate(str.maketrans('', '', punc)) for t in dem]
    dem_tokenized = [token.text for t in dem for token in nlp(t)]
    dem_tokenized = [t for t in dem_tokenized if t not in all_screenname and t not in exclude_words]
    rep = [t.replace('\n', '').replace('&amp', 'and').translate(str.maketrans('', '', punc)) for t in rep]
    rep_tokenized = [token.text for t in rep for token in nlp(t)]
    rep_tokenized = [t for t in rep_tokenized if t not in all_screenname and t not in exclude_words]
    bg = [t.replace('\n', '').replace('&amp', 'and').translate(str.maketrans('', '', punc)) for t in bg]
    bg_tokenized = [token.text for t in bg for token in nlp(t)]
    bg_tokenized = [t for t in bg_tokenized if t not in all_screenname and t not in exclude_words]

    dem_counter=  Counter(dem_tokenized)
    rep_counter= Counter(rep_tokenized)
    bg_counter = Counter(bg_tokenized)
    return dem_counter, rep_counter, bg_counter

In [17]:
dem_counter, rep_counter, bg_counter = preprocess_text(dem_covid, rep_covid, bg)

the health of american people &amp; u.s. economy must come first. instead of forcing #unemployment aid to lapse while republicans fight among themselves, senate gop should join dems in extending #ui to save people &amp; businesses as covid cases are surging. 



In [18]:
len(bg_counter), len(dem_counter), len(rep_counter)

(183372, 52737, 36269)

In [19]:
dem_counter['covid']

63371

In [20]:
rep_counter['covid']

38787

In [21]:
bg_counter['covid']

0

In [22]:
dem_counter = {k:v for k, v in dem_counter.items() if v > 2 }
rep_counter = {k:v for k, v in rep_counter.items() if v > 2 }
bg_counter = {k:v for k, v in bg_counter.items() if v > 2 }


In [23]:
dem_counter

{'the': 84700,
 'health': 11041,
 'of': 36770,
 'american': 1665,
 'people': 4016,
 'and': 58384,
 'us': 4537,
 'economy': 1174,
 'must': 5092,
 'come': 497,
 'first': 1954,
 'instead': 286,
 'forcing': 55,
 '#': 18779,
 'unemployment': 1331,
 'aid': 832,
 'to': 84301,
 'lapse': 4,
 'while': 1436,
 'republicans': 333,
 'fight': 2362,
 'among': 320,
 'themselves': 288,
 'senate': 1395,
 'gop': 192,
 'should': 2781,
 'join': 2925,
 'dems': 99,
 'in': 28911,
 'extending': 85,
 'ui': 26,
 'save': 984,
 'businesses': 3347,
 'as': 8979,
 'covid': 63371,
 'cases': 3675,
 'are': 16099,
 'surging': 67,
 'rt': 12719,
 'cdcgov': 1317,
 'can': 7369,
 'be': 8536,
 'spread': 4773,
 'by': 6983,
 'who': 5140,
 'do': 4535,
 'not': 4960,
 'have': 10964,
 'symptoms': 894,
 'know': 1557,
 'that': 12197,
 'they': 5278,
 'infected': 224,
 'cloth': 164,
 'face': 1411,
 'coverings': 132,
 'may': 1888,
 'help': 7240,
 'p': 75,
 '…': 10503,
 'habitatorg': 3,
 'our': 18689,
 'live': 3137,
 'discussion': 415,
 'a

In [28]:
dem_counter_df = pd.DataFrame.from_dict(dem_counter, orient='index').reset_index()
dem_counter_df.columns = ['token', 'dem_freq']

rep_counter_df = pd.DataFrame.from_dict(rep_counter, orient='index').reset_index()
rep_counter_df.columns = ['token', 'rep_freq']

bg_counter_df = pd.DataFrame.from_dict(bg_counter, orient='index').reset_index()
bg_counter_df.columns = ['token', 'bg_freq']

In [34]:
rep_counter_df['rep_rank'] = rep_counter_df['rep_freq'].rank(method='dense', ascending=False).astype(int)

In [37]:
dem_counter_df['dem_rank'] = dem_counter_df['dem_freq'].rank(method='dense', ascending=False).astype(int)

In [38]:
bg_counter_df['bg_rank'] = bg_counter_df['bg_freq'].rank(method='dense', ascending=False).astype(int)

In [40]:
temp = pd.merge(dem_counter_df, rep_counter_df, on='token', how='inner')

In [41]:
len(dem_counter_df), len(rep_counter_df), len(temp)

(17887, 13390, 9302)

In [42]:
all_counter_df = pd.merge(temp, bg_counter_df, how='left', on='token')

In [43]:
len(all_counter_df)

9302

In [44]:
all_counter_df.head(2)

,token,dem_freq,dem_rank,rep_freq,rep_rank,bg_freq,bg_rank
0,the,84700,1,52382,1,413168.0,1.0
1,health,11041,21,4201,33,24732.0,45.0


In [46]:
all_counter_df['dem_rep_diff'] = np.abs(all_counter_df['dem_rank'] - all_counter_df['rep_rank'])

In [48]:
all_counter_df['dem_bg_diff'] = np.abs(all_counter_df['dem_rank'] - all_counter_df['bg_rank'])

In [49]:
all_counter_df['rep_bg_diff'] = np.abs(all_counter_df['rep_rank'] - all_counter_df['bg_rank'])

In [50]:
all_counter_df.head(2)

,token,dem_freq,dem_rank,rep_freq,rep_rank,bg_freq,bg_rank,dem_rep_diff,dem_bg_diff,rep_bg_diff
0,the,84700,1,52382,1,413168.0,1.0,0,0.0,0.0
1,health,11041,21,4201,33,24732.0,45.0,12,24.0,12.0


In [53]:
all_counter_df = all_counter_df.fillna(0)

In [59]:
dem_bg = all_counter_df.sort_values(by='dem_bg_diff', ascending=False).head(100)

In [64]:
dem_bg.head(10).token.tolist()

['outbreak',
 'novel',
 'symptoms',
 'hospitalized',
 'related',
 'vaccine',
 'essex',
 'beshear',
 'presumptive',
 'teletown']

In [60]:
rep_bg = all_counter_df.sort_values(by='rep_bg_diff', ascending=False).head(100)

In [66]:
rep_bg.head(11).token.tolist()

['novel',
 'outbreak',
 'msdh',
 'wvdhhr',
 'azdhs',
 'hospitalized',
 'recovered',
 'plasma',
 'hysteria',
 '📽',
 'preparations']

In [68]:
pd.merge(dem_bg, rep_bg, on='token', how='inner').token.tolist()[0:10]

['outbreak',
 'novel',
 'symptoms',
 'hospitalized',
 'presumptive',
 'recovered',
 'plasma',
 'shortness',
 'spreads',
 'slowing']